In [ ]:
def get_data():
    import keras
    from keras.datasets import cifar10
    import numpy as np

    num_classes = 10
    # the function_str, split between train and test sets
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x = np.concatenate((x_train, x_test))
    y = np.concatenate((y_train, y_test))

    # convert class vectors to binary class matrices
    predictors = x.astype('float32')
    labels = keras.utils.to_categorical(y, num_classes)

    print(predictors.shape)

    return predictors, labels

def divide_data(x, t, num_sets):
    import numpy as np
    train_x_sets = np.split(x, num_sets)
    train_t_sets = np.split(t, num_sets)
    
    return train_x_sets, train_t_sets
    
def process_data(predictors, labels, train_percentage, validation_percentage):
    num_samples = len(predictors)
    num_train_samples = int(num_samples * train_percentage)
    num_validation_samples = int(num_samples * validation_percentage)

    x_train, y_train = predictors[:num_train_samples], labels[:num_train_samples]
    x_validation, y_validation = predictors[num_train_samples:num_train_samples + num_validation_samples], \
                                 labels[num_train_samples:num_train_samples + num_validation_samples]
    x_test, y_test = predictors[num_train_samples + num_validation_samples:], \
                     labels[num_train_samples + num_validation_samples:]


    input_shape = x_train.shape[1:]
    data_dict = {
            "x_train": x_train,
            "x_test": x_test,
            "y_train": y_train,
            "y_test": y_test,
            "x_validation": x_validation,
            "y_validation": y_validation,
            "input_shape": input_shape
        }

    return data_dict

In [119]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, Activation, Concatenate
    
def create_base_model(input_shape, num_classes, name):
    input_layer = Input(shape=input_shape, name=name)
    conv = Conv2D(32, kernel_size=(3, 3),
                     activation='relu')(input_layer)
    max_pool_1 = MaxPooling2D(pool_size=(2, 2))(conv)
    dropout_1 = Dropout(0.25)(max_pool_1)
    flatten = Flatten()(dropout_1)
    dense = Dense(2, activation='relu')(flatten)
    dropout_2 = Dropout(0.5)(dense)
    output = Dense(num_classes, activation='softmax')(dropout_2)
    
    model = Model(inputs=[input_layer], outputs=[output], name=name)
    
    return model

def create_keras_web_model(input_shape, num_classes, name):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    
    return model

def create_combining_model(input_layers, num_classes):
    input_layer = Input(shape=input_shape)
    dropout_1 = Dropout(0.25)(max_pool_1)
    flatten = Flatten()(dropout_1)
    dense = Dense(32, activation='relu')(flatten)
    dropout_2 = Dropout(0.5)(dense)
    output = Dense(num_classes, activation='softmax')(dropout_2)
    
    model = Model(inputs=[input_layer], outputs=[output], name='combining_model')
    return model

def collect_output_tensors(models):
    output_tensors = []
    for model in models:
        output_tensors.append(model.output)
        
    return output_tensors

def collect_input_tensors(models):
    input_tensors = []
    for model in models:
        input_tensors.append(model.input)
        
    return input_tensors

def create_combination(models, num_classes):
    input_tensors = collect_input_tensors(models)
    output_tensors = collect_output_tensors(models)
    concat_layer = concatenate(output_tensors, axis=-1)
    dense1 = Dense(32)(concat_layer)
    output = Dense(num_classes, activation='softmax')(dense1)
    
    model = Model(inputs=input_tensors, 
                  outputs=[output], 
                  name="combining model")
    
    return model

In [44]:
x, t = get_data()

train_percentage = 0.001
validation_percentage = 0.1
data_dict = process_data(x, t, train_percentage, validation_percentage)

train_x = data_dict['x_train']
train_t = data_dict['y_train']
test_x = data_dict['x_test']
test_t = data_dict['y_test']
validation_x = data_dict['x_validation']
validation_t = data_dict['y_validation']


num_train_sets = 2
train_x_sets, train_t_sets = divide_data(train_x, train_t, num_train_sets)

(60000, 32, 32, 3)


In [59]:
data_input_shape = data_dict['input_shape']
num_classes = 10

In [60]:
import keras
models = []
for model_idx in range(num_train_sets):
    model = create_keras_web_model(data_input_shape, 
                                   num_classes, 
                                   f"ciphar_model_{model_idx}")
    models.append(model)

In [61]:
for model in models:
    model.compile(loss='binary_crossentropy',
                                   optimizer=keras.optimizers.Adadelta(),
                                   metrics=['accuracy'])
    model.fit(train_x_sets[model_idx], train_t_sets[model_idx],
                           batch_size=32,
                           epochs=1,
                           verbose=True,
                           validation_data=(test_x, test_t))

Train on 30 samples, validate on 53940 samples
Epoch 1/1


KeyboardInterrupt: 

In [ ]:
for model in models:
    scores = model.evaluate(x, t, verbose=True)
    print("Test Score: ", scores[1])

In [ ]:
models[1].layers[-1].output_shape

In [108]:
models[1].input

<tf.Tensor 'conv2d_22_input:0' shape=(?, 32, 32, 3) dtype=float32>

In [120]:
combined_model = create_combination(models, num_classes)